In [54]:
import os
from pathlib import Path
import pandas as pd
import requests
from bs4 import BeautifulSoup, SoupStrainer

from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys

import time

In [55]:
DATA_PATH = Path.cwd().parent.parent / "data" / "raw"

In [56]:
recipes_urls = pd.read_csv(DATA_PATH / "urls.csv")
recipes_urls = recipes_urls['url'].to_list()

## Define function to load data

need to make https://www.veganrecipeclub.org.uk/recipes/15-minute-flourless-choc-chip-banana-muffins/ work

### Find the different section of the web page

In [57]:
#Title
def get_title(soup):
    return soup.find_all("h1")[0].get_text(strip=True)

In [58]:
#section name
def get_sections(soup):
    elements = dict()
    sections = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})
    for i, section in enumerate(sections):
        elements[section.text] = i
    return elements

In [59]:
#Preparation time
def get_preparation_time(soup, elements):
    time = 0
    html_element = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})

    try:
        if(html_element[elements['Preparation time']].nextSibling.nextSibling.text != None):
            time = html_element[elements['Preparation time']].nextSibling.nextSibling.text
    except:
        return 0

    return time

#Cooking time
def get_cooking_time(soup, elements):
    time = 0
    html_element = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})

    try:
        if(html_element[elements['Cooking time']].nextSibling.nextSibling.text != None):
            time = html_element[elements['Cooking time']].nextSibling.nextSibling.text
    except:
        return 0

    return time

#Preparation time
def get_total_time(soup, elements):
    time = 0
    html_element = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})

    try:
        if(html_element[elements['Total time']].nextSibling.nextSibling.text != None):
            time = html_element[elements['Total time']].nextSibling.nextSibling.text
    except:
        return 0

    return time

In [72]:
#Style of cooking
def get_cooking_style(soup):
    cooking_styles = ''
    styles = soup.find_all("p", {"class": "intList"})
    if len(styles) != 0:
        for style in styles:
            cooking_styles += style.get_text(strip=True) + ' '
    return cooking_styles

In [73]:
#Type of recipe
def get_recipe_type(soup, elements):
    html_element = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})
    recipe_types = ''
    try:
        types = html_element[elements['Meal']].nextSibling.nextSibling.find_all("li")
        for type_ in types:
            recipe_types += type_[0] + ' '
    except:
        return ''
    return recipe_types

In [62]:
#Number of servings
def get_number_servings(soup, elements):
    html_element = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})
    try:
        return html_element[elements['Servings']].nextSibling.nextSibling.get_text()
    except:
        return 'unknown'

In [63]:
#Ingredients
def get_ingredients(soup, elements):
    html_element = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})
    ingredients = []
    try:
        HTML_ingredients = html_element[elements['Ingredients']].nextSibling.nextSibling
        for item in HTML_ingredients.find_all("li"):
            ingredients.append(item.get_text())

        if(len(HTML_ingredients.find_all("li")) == 0):
            #Ingredients
            HTML_ingredients = HTML_ingredients[elements['Ingredients']].parent.find("div", {"class": "recipe-list"}).p
            for ingredient in HTML_ingredients.get_text().split("\n"):
                ingredients.append(ingredient)
    except:
        return []
    return ingredients

In [64]:
#Instructions
def get_instructions(soup, elements):
    html_element = soup.find_all("p", {"style": "text-align: center; font-weight: bold;"})
    instructions = []
    try: 
        HTML_instructions = html_element[elements['Instructions']].nextSibling.nextSibling
        for item in HTML_instructions.find_all("li"):
            instructions.append(item.get_text())
    except:
        return []
    return instructions

## Scrape every urls

In [65]:
recipes_urls[:5]

['https://www.veganrecipeclub.org.uk/recipes/beetroot-dhal/',
 'https://www.veganrecipeclub.org.uk/recipes/avocado-wings-with-ranch-dip/',
 'https://www.veganrecipeclub.org.uk/recipes/black-forest-gateau-smoothie/',
 'https://www.veganrecipeclub.org.uk/recipes/super-fast-healthy-ice-cream/',
 'https://www.veganrecipeclub.org.uk/recipes/52-fast-day-spiced-tomato-dahl/']

In [85]:
%%time
rows = []
columns = ['title', 'preparation_time', 'cooking_time', 'total_time', 'style', 'type', 'servings', 'ingredients', 'instructions']
for url in recipes_urls[:50]:
    time.sleep(5)
    row = []
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    sections = get_sections(soup)
    
    row.append(get_title(soup))
    row.append(get_preparation_time(soup, sections))
    row.append(get_cooking_time(soup, sections))
    row.append(get_total_time(soup, sections))
    row.append(get_cooking_style(soup))
    row.append(get_recipe_type(soup, sections))
    row.append(get_number_servings(soup, sections))
    row.append(get_ingredients(soup, sections))
    row.append(get_instructions(soup, sections))
    
    
    rows.append(row)
    
df = pd.DataFrame(rows, columns = columns)

Wall time: 5min 41s


In [86]:
df

,title,preparation_time,cooking_time,total_time,style,type,servings,ingredients,instructions
0,Beetroot Dhal,,0,60,Indian and Indian Subcontinent,,2-4,"[2 beetroot, chopped into small cubes , Drizzl...","[Preheat your oven to 180°C/350°F/Gas Mark 4.,..."
1,Avocado Wings with Ranch Dip,,0,20,,,4,"[500ml high-quality soy milk, 42g lemon juice,...",[Combine all the ingredients (except the oil) ...
2,Black Forest Gateau Smoothie,,0,5 minutes,,,2,"[160g/1 cup frozen berries of your choice, 1 v...",[Blend all ingredients until smooth.]
3,Super-fast Healthy Ice Cream,,0,5-7 minutes,,,4-6,"[90g/⅔ cup raw, unsalted Brazil nuts, 8-9 fres...","[Blend the Brazil nuts, dates, water, and vani..."
4,5:2 Fast Day – Spiced Tomato Dahl,,0,40 minutes,Indian and Indian Subcontinent,,1,"[2 squirts oil spray (2 cals), 80g red onion, ...",[]
5,Sausages with Apple Mustard Mash & Gravy,,0,30 minutes,British,,4,"[2 tbsp vegetable oil, 2 red onions, 1 large s...","[If making home-made sausages, prepare and sha..."
6,Punk Pumpkin Cake,1 hr,0,110,,,10,"[100g gram flour, 50g potato starch or tapioca...",[Preheat your oven to 200c/390f/GM6. Grease th...
7,Vegan Chocolate Brownie,,0,0,,,unknown,"[5 tbsp coconut oil, melted, 200g vegan dark c...","[Preheat the oven to 180ºC/350ºF/Gas Mark 4., ..."
8,Store Cupboard Nut Roast,,0,45,,,4,"[1 tbsp vegetable oil, 1 onion, finely chopped...",[Preheat the oven to 180°C (fan) 356°F/Gas Mar...
9,Cashew Cream & Beetroot Crostini,,0,0,,,6,"[200g cashew nuts, soaked for minimum of 2 hou...",[Blend all of the ingredients together until s...


## Transform columns of dataframe

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             10 non-null     object
 1   preparation_time  10 non-null     object
 2   cooking_time      10 non-null     int64 
 3   total_time        10 non-null     object
 4   style             10 non-null     object
 5   type              10 non-null     object
 6   servings          10 non-null     object
 7   ingredients       10 non-null     object
 8   instructions      10 non-null     object
dtypes: int64(1), object(8)
memory usage: 848.0+ bytes


In [83]:
for col in ['preparation_time', 'cooking_time', 'total_time']:
    df[col] = df[col].fillna(0)
    df[col] = df[col].astype(str)
    df[col] = df[col].str.extract('(\d+)', expand=False)

In [84]:
df

,title,preparation_time,cooking_time,total_time,style,type,servings,ingredients,instructions
0,Beetroot Dhal,0,0,60,Indian and Indian Subcontinent,,2-4,"[2 beetroot, chopped into small cubes , Drizzl...","[Preheat your oven to 180°C/350°F/Gas Mark 4.,..."
1,Avocado Wings with Ranch Dip,0,0,20,,,4,"[500ml high-quality soy milk, 42g lemon juice,...",[Combine all the ingredients (except the oil) ...
2,Black Forest Gateau Smoothie,0,0,5,,,2,"[160g/1 cup frozen berries of your choice, 1 v...",[Blend all ingredients until smooth.]
3,Super-fast Healthy Ice Cream,0,0,5,,,4-6,"[90g/⅔ cup raw, unsalted Brazil nuts, 8-9 fres...","[Blend the Brazil nuts, dates, water, and vani..."
4,5:2 Fast Day – Spiced Tomato Dahl,0,0,40,Indian and Indian Subcontinent,,1,"[2 squirts oil spray (2 cals), 80g red onion, ...",[]
5,Sausages with Apple Mustard Mash & Gravy,0,0,30,British,,4,"[2 tbsp vegetable oil, 2 red onions, 1 large s...","[If making home-made sausages, prepare and sha..."
6,Punk Pumpkin Cake,1,0,110,,,10,"[100g gram flour, 50g potato starch or tapioca...",[Preheat your oven to 200c/390f/GM6. Grease th...
7,Vegan Chocolate Brownie,0,0,0,,,unknown,"[5 tbsp coconut oil, melted, 200g vegan dark c...","[Preheat the oven to 180ºC/350ºF/Gas Mark 4., ..."
8,Store Cupboard Nut Roast,0,0,45,,,4,"[1 tbsp vegetable oil, 1 onion, finely chopped...",[Preheat the oven to 180°C (fan) 356°F/Gas Mar...
9,Cashew Cream & Beetroot Crostini,0,0,0,,,6,"[200g cashew nuts, soaked for minimum of 2 hou...",[Blend all of the ingredients together until s...
